# Imports

In [ ]:
import glob
import itertools as it
import os
from functools import partial
from pathlib import Path

import dask
import distributed
import h5py
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import scipy
import skimage.measure
import zarr
from dask import delayed
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster, progress
from holoviews.operation.datashader import regrid
from tqdm.auto import tqdm, trange

IDX = pd.IndexSlice

In [ ]:
# TODO: does this help?
# %config InlineBackend.figure_format = "jpg"

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.image_analysis.calibration.distortion as distortion
from paulssonlab.image_analysis import *
from paulssonlab.image_analysis.ui import display_image

In [ ]:
%load_ext pyinstrument

In [ ]:
hv.extension("bokeh")

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="06:00:00",
    memory="2GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/log",
    cores=1,
    processes=1,
)
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(1)

In [ ]:
cluster.adapt(maximum=20)

# Geometric distortion

## Image correction is equivalent to coördinate correction

In [ ]:
filename = "/home/jqs1/scratch/jqs1/microscopy/230728/calibration/230728_ultrarainbow_64fov_cfp.nd2"
# filename = "/home/jqs1/scratch/jqs1/microscopy/230728/calibration/230728_ultrarainbow_64fov_cy5.nd2"
nd2 = nd2reader.ND2Reader(filename)

In [ ]:
%%time
k1 = 7e-10
img_distorted = nd2.get_frame_2D(v=fov1, t=t, z=z_idx, c=channel_idx)
coords_distorted = distortion.df_to_coords(distortion.find_puncta(img_distorted))
input_shape = img_distorted.shape
input_center = image.center_from_shape(input_shape)
output_shape, output_center = image.radial_distortion_output(
    k1, input_shape, input_center
)
# output_shape not used
coords_distorted_corrected = image.radial_distortion(
    coords_distorted, k1=k1, input_center=input_center, output_center=output_center
)
img_corrected = image.correct_radial_distortion(img_distorted, k1=k1)
coords_corrected = distortion.df_to_coords(distortion.find_puncta(img_corrected))

In [ ]:
correspondence_dists, correspondence_idxs = distortion.nearest_neighbors(
    coords_corrected, coords_distorted_corrected
)

In [ ]:
plt.hist(correspondence_dists, bins=100, log=True);

In [ ]:
plt.figure(figsize=(30, 30), dpi=200)
distortion.plot_puncta(
    img=img_corrected,
    coords=coords_corrected,
    coords2=coords_distorted_corrected,
    scale=0.99,
)

## Old

In [ ]:
# Cy7 translation, GFP/YFP translations
# better SNR Cy5?
# measure focus offset in PFS units, verify

In [ ]:
# filename = "/home/jqs1/scratch/jqs1/microscopy/230726/calibration/230726_ultrarainbow_40x40_Cy5-EM.nd2"
filename = "/home/jqs1/scratch/jqs1/microscopy/230728/calibration/230728_ultrarainbow_64fov_cfp.nd2"
# filename = "/home/jqs1/scratch/jqs1/microscopy/230728/calibration/230728_ultrarainbow_64fov_cy5.nd2"
# filename = "/home/jqs1/scratch/jqs1/microscopy/230726/calibration/230726_ultrarainbow_40x40_zstack_nocy7_fov1.nd2"
nd2 = nd2reader.ND2Reader(filename)

In [ ]:
nd2.metadata["channels"]

In [ ]:
nd2.sizes

In [ ]:
%%time
img1_orig = nd2.get_frame_2D(v=0, t=0, z=0, c=0)  # [:500,:500]
img2_orig = nd2.get_frame_2D(v=1, t=0, z=0, c=0)  # [:500,:500]
# k1 = 5.199999999997459e-10
k1 = 8.947368421052635e-10
img1 = image.correct_radial_distortion(
    nd2.get_frame_2D(v=0, t=0, z=0, c=0), k1=k1
)  # [:500,:500]
img2 = image.correct_radial_distortion(
    nd2.get_frame_2D(v=1, t=0, z=0, c=0), k1=k1
)  # [:500,:500]

In [ ]:
%%time
df1, img_labels = distortion.find_puncta(img1, filter=True, return_labels=True)

In [ ]:
%%time
df2 = distortion.find_puncta(img2, filter=True)

In [ ]:
%%time
df1_orig = distortion.find_puncta(img1_orig)
df2_orig = distortion.find_puncta(img2_orig)

In [ ]:
plt.figure(figsize=(30, 30), dpi=200)
df1_orig_trans = distortion.translate_df(df1_orig, (-48, 0))
distortion.plot_puncta(img=img2_orig, df=df1_orig_trans, df2=df2_orig, scale=0.999)

In [ ]:
plt.figure(figsize=(30, 30), dpi=200)
distortion.plot_puncta(img=img1, labels=None, df=df1, scale=0.999)

In [ ]:
plt.figure(figsize=(30, 30), dpi=200)
df_trans = distortion.translate_df(df, (-48, 0))
distortion.plot_puncta(img=img2, df=df_trans, df2=df2, scale=0.999)

In [ ]:
plt.figure(figsize=(30, 30), dpi=200)
distortion.plot_puncta(img=img1, labels=None, df=df, scale=0.999)

In [ ]:
plt.figure(figsize=(30, 30), dpi=200)
df_trans = distortion.translate_df(df, (-48, 0))
distortion.plot_puncta(img=img2, df=df_trans, scale=0.999)

In [ ]:
%%time
df2, img_labels = distortion.find_puncta(img2, filter=True, return_labels=True)

In [ ]:
df_trans = distortion.translate_df(df, (-48, 0))
nn = distortion.nearest_neighbors_df(df_trans, df2)

In [ ]:
plt.hist(nn[0], bins=100, log=True);

In [ ]:
# INPUT: use stage x/y to guess initial translations/rotations (0) for each FOV pair

# LOOP until converged:
# FOR EACH FOV PAIR: compute nearest neighbor correspondences
# FOR EACH FOV PAIR: optimize translation/rotation for fixed coordinates
# FOR ALL FOV PAIRS JOINTLY: optimize distortion params

In [ ]:
xs = np.asarray(nd2._parser._raw_metadata.x_data)
ys = np.asarray(nd2._parser._raw_metadata.y_data)

In [ ]:
# init
max_correspondence_dist = 5
t = 0
z_idx = 0
channel_idx = 0
coords = {}

In [ ]:
pairs = it.pairwise(range(3))

In [ ]:
# fov1, fov2 = (0, 1)
fov1, fov2 = (6, 7)

In [ ]:
%%time
if (coords1 := coords.get(fov1)) is None:
    img = nd2.get_frame_2D(v=fov1, t=t, z=z_idx, c=channel_idx)
    shape = img.shape
    coords1 = distortion.df_to_coords(distortion.find_puncta(img))
    del img
    coords[fov1] = coords1
if (coords2 := coords.get(fov2)) is None:
    img = nd2.get_frame_2D(v=fov2, t=t, z=z_idx, c=channel_idx)
    coords2 = distortion.df_to_coords(distortion.find_puncta(img))
    del img
    coords[fov2] = coords2

In [ ]:
stage_offset = np.array(
    [
        xs[fov2] - xs[fov1],
        ys[fov2] - ys[fov1],
    ]
)
objective_magnification = 20
pixel_size = 4.25
translation_guess = stage_offset * objective_magnification / pixel_size
translation_guess

In [ ]:
skimage.transform.

In [ ]:
tform = skimage.transform.EuclideanTransform(translation=translation_guess)
# TODO: try TranslationTransform?

In [ ]:
coords2_transformed = tform.inverse(coords2)

In [ ]:
correspondence_dists, correspondence_idxs = distortion.nearest_neighbors(
    coords1, coords2_transformed
)

In [ ]:
correspondence_mask = correspondence_dists < max_correspondence_dist
coords1_correspondence = coords1[correspondence_mask]
coords2_correspondence = coords2[correspondence_idxs][correspondence_mask]

In [ ]:
plt.hist(correspondence_dists[correspondence_dists < 5], bins=100, log=True);

In [ ]:
tform.estimate(coords1_correspondence, coords2_correspondence)

In [ ]:
(tform.translation, tform.rotation)

In [ ]:
# TODO: is this right or off-by-one?
# center = (np.array(shape[::-1]) - 1) / 2
center = None
k1 = 8.947368421052635e-10
coords1_undistorted = image.radial_distortion(
    coords1_correspondence, input_center=center, k1=k1
)
coords2_undistorted = image.radial_distortion(
    tform.inverse(coords2_correspondence), input_center=center, k1=k1
)

In [ ]:
np.sqrt(((coords1_undistorted - coords2_undistorted) ** 2).mean())

In [ ]:
coords1_correspondence

In [ ]:
coords1_undistorted

In [ ]:
vectorfield_difference(coords1_undistorted, coords2_undistorted)

In [ ]:
vectorfield_difference(coords1_correspondence, coords1_undistorted)

In [ ]:
# TODO: is this right or off-by-one?
k1s = np.arange(-1e-8, 1e-8, 1e-11)
rmses = []
for k1 in tqdm(k1s):
    coords1_undistorted = image.radial_distortion(
        coords1_correspondence, input_center=center, k1=k1
    )
    coords2_undistorted = tform.inverse(
        image.radial_distortion(coords2_correspondence, input_center=center, k1=k1)
    )
    rms = np.sqrt(((coords1_undistorted - coords2_undistorted) ** 2).mean())
    rmses.append(rms)

In [ ]:
plt.plot(k1s, rmses);

In [ ]:
k1s[np.argmin(rmses)]

In [ ]:
k1s[np.argmin(rmses)]

In [ ]:
k1 = 5.199999999997459e-8
img_undistorted = nd2.get_frame_2D(v=0, t=0, z=0, c=0)
img_distorted = image.correct_radial_distortion(img_undistorted, k1=k1)

In [ ]:
%%time
input_center = (500, 500)
img_distorted2 = image.correct_radial_distortion(
    img_undistorted, k1=k1, input_center=input_center
)

In [ ]:
img_distorted2

In [ ]:
display_image(img_distorted2, scale=0.99)

In [ ]:
display_image(img_distorted, scale=0.99)

In [ ]:
display_image(img_undistorted, scale=0.99)

In [ ]:
img_undistorted = nd2.get_frame_2D(v=0, t=0, z=0, c=0)

In [ ]:
img_undistorted.shape

In [ ]:
k1s = np.linspace(1e-10, 3e-9, 30)
# k1 = 5.199999999997459e-10
xs = np.linspace(0, img_undistorted.shape[1] - 1, 10)
ys = np.linspace(0, img_undistorted.shape[0] - 1, 10)
rmses = np.zeros((len(k1s), len(ys), len(xs)))
for x_idx, x in enumerate(tqdm(xs)):
    for y_idx, y in enumerate(ys):
        for k1_idx, k1 in enumerate(k1s):
            center = (x, y)
            coords1_undistorted = image.radial_distortion(
                coords1_correspondence, input_center=center, k1=k1
            )
            coords2_undistorted = tform.inverse(
                image.radial_distortion(
                    coords2_correspondence, input_center=center, k1=k1
                )
            )
            rms = np.sqrt(((coords1_undistorted - coords2_undistorted) ** 2).mean())
            rmses[k1_idx, y_idx, x_idx] = rms

In [ ]:
# good: [2527.5, 1979.5]

In [ ]:
(xs[5], ys[5])

In [ ]:
k1s[4]

In [ ]:
plt.plot(rmses[:, 5, 5])
plt.plot(rmses[:, 4, 4]);

In [ ]:
plt.plot(rmses[0, :, 5]);

In [ ]:
plt.imshow(rmses[2]);

# Trench detection

In [ ]:
width_to_pitch_ratio = 1.4 / 3.5
segmentation_channel = "RFP-EM"
filename = workflow.SplitFilename(
    sorted(
        glob.glob(
            # "/home/jqs1/scratch/jqs1/microscopy/230707/230707_repressilators_restart.nd2.split.a*"
            "/home/jqs1/scratch/jqs1/microscopy/230830/230830_repressilators.nd2.split.*"
        )
    )
)

In [ ]:
def get_frame_func(
    filename, position, channel, t, k1=0, center=None, dark=None, flat=None
):
    return image.correct_radial_distortion(
        np.asarray(
            workflow.get_nd2_frame(
                filename, position=position, channel=channel, t=t, dark=dark, flat=flat
            )
        ),
        k1=k1,
        input_center=center,
    )  # [600:2400, 1500:3500]
    # return np.asarray(
    #     workflow.get_nd2_frame(filename, position, channel, t, dark=dark, flat=flat)
    # )[550:2350, 1500:3500]

In [ ]:
display_image(img0, scale=0.99, downsample=4)

In [ ]:
img_uncorrected = (
    nd2.get_frame_2D()
)  # get_frame_func(filename, 258, segmentation_channel, 0, k1=0)

In [ ]:
img_uncorrected.shape

In [ ]:
display_image(img0, downsample=1, scale=0.99)

In [ ]:
center = image.center_from_shape(img_uncorrected.shape) - np.array([0, -500])

In [ ]:
center

In [ ]:
%%time
# k1 = 5.199999999997459e-10
# k1 = 7e-10
# k1 = 6.8e-10
k1 = 1e-9
img0 = get_frame_func(filename, 0, segmentation_channel, 0, k1=k1, center=center)
image_limits = geometry.get_image_limits(img0.shape)

In [ ]:
%%time
diag = util.tree()
rois, info = trench_detection.find_trenches(
    img0,
    width_to_pitch_ratio=width_to_pitch_ratio,
    join_info=False,
    diagnostics=diag,
)
angle = info["angle"]
pitch = info["pitch"]

In [ ]:
# k1=
(angle, pitch)

In [ ]:
# k1=5e-10
(angle, pitch)

In [ ]:
# k1=0
(angle, pitch)

In [ ]:
# k1=1e-9, -500
diag["labeling"]["set_finding"]["image_with_lines"]

In [ ]:
# k1=1e-9, -500
diag["labeling"]["set_finding"]["image_with_lines"]

In [ ]:
# k1=1e-9, +1000
diag["labeling"]["set_finding"]["image_with_lines"]

In [ ]:
# k1=1e-9, -1000
diag["labeling"]["set_finding"]["image_with_lines"]

In [ ]:
# k1=1e-10
diag["labeling"]["set_finding"]["image_with_lines"]

In [ ]:
# k1=8e-10
diag["labeling"]["set_finding"]["image_with_lines"]

In [ ]:
# k1=
diag["labeling"]["set_finding"]["image_with_lines"]

In [ ]:
# k1=0
diag["labeling"]["set_finding"]["image_with_lines"]

In [ ]:
diag["bboxes"]

In [ ]:
trench_detection.plot_trenches(bboxes=False, lines=True)

In [ ]:
diag["labeling"]["find_periodic_lines"]["profile"]  # .keys()

In [ ]:
diag["labeling"]["find_periodic_lines"]["profile"]  # .keys()

In [ ]:
diag["labeling"]["find_periodic_lines"]["profile"]  # .keys()

In [ ]:
diag["labeling"]["find_periodic_lines"]["profile"]  # .keys()

In [ ]:
diag["labeling"]["find_periodic_lines"]["profile"]  # .keys()

In [ ]:
(angle, pitch)

In [ ]:
diag["labeling"]["find_periodic_lines"]["peak_func"]["spectrum"]

In [ ]:
diag["labeling"]["find_periodic_lines"]["profile"]  # .keys()

In [ ]:
diag["labeling"]["set_finding"]["image_with_lines"]

In [ ]:
diag["bboxes"]

In [ ]:
diag["labeling"]["set_finding"]["profiles"]